In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/layer9/valid.csv
/kaggle/input/layer9/train.csv
/kaggle/input/layer9/test.csv


In [2]:
import pandas as pd
import numpy as np
from pandas import Series

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
train_df = pd.read_csv('/kaggle/input/layer9/train.csv')
valid_df = pd.read_csv('/kaggle/input/layer9/valid.csv')
test_df = pd.read_csv('/kaggle/input/layer9/test.csv')

In [4]:
train_df.shape

(28520, 772)

In [5]:
train_df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.019301,0.059756,0.081375,0.057481,-0.068440,-0.165913,0.035643,-0.091138,0.021688,0.057158,...,-0.035576,0.127319,0.098128,-0.058787,0.100971,-0.047754,45,NaN,1,6
1,0.049741,0.090030,0.035118,-0.013676,-0.194317,-0.101763,0.085875,-0.081317,0.112418,0.120523,...,0.020538,0.058968,0.029803,0.111324,0.036727,0.031927,45,NaN,1,6
2,0.019212,0.087779,0.093907,-0.033738,-0.141409,-0.062881,-0.071402,-0.006599,0.020372,-0.027777,...,0.119645,-0.040861,0.000548,-0.061003,-0.042450,0.063340,45,NaN,1,6
3,0.070283,0.049040,0.042126,0.122637,-0.056964,-0.113700,0.108454,0.051336,0.086610,0.141578,...,-0.124494,-0.169225,-0.046391,0.148787,0.014616,-0.140644,45,NaN,1,6
4,0.028864,0.165634,0.016302,0.036117,-0.028871,-0.147748,0.053180,0.025071,-0.004200,-0.022183,...,-0.124862,0.044907,0.084005,-0.038450,0.084371,-0.072146,45,NaN,1,6


In [6]:
missing_columns = train_df.columns[train_df.isnull().any()]
missing_counts = train_df[missing_columns].isnull().sum()

print('Missing Columns and Counts')
for column in missing_columns:
    print( str(column) +' : '+ str(missing_counts[column]))

Missing Columns and Counts
label_2 : 480


In [7]:
train_data = train_df.copy()
valid_data = valid_df.copy()
test_data = test_df.copy()

In [8]:
train_df.describe()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
count,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,...,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28040.000000,28520.000000,28520.000000
mean,-0.015964,0.075453,0.045892,0.046230,-0.068548,-0.090882,0.034405,-0.038091,0.049623,0.036286,...,-0.058015,0.017027,-0.002970,0.053544,0.033636,0.003864,30.498843,27.975107,0.799299,5.997125
std,0.048122,0.047099,0.059531,0.056354,0.062569,0.054895,0.065229,0.058487,0.051186,0.060626,...,0.075779,0.055990,0.055413,0.082085,0.070793,0.059617,17.328389,5.735913,0.400532,2.375567
min,-0.300831,-0.113586,-0.183329,-0.212180,-0.385634,-0.331156,-0.320988,-0.519168,-0.171353,-0.404408,...,-0.343703,-0.401326,-0.355695,-0.390809,-0.470045,-0.242932,1.000000,22.000000,0.000000,0.000000
25%,-0.048373,0.046609,0.004613,0.007529,-0.109321,-0.128968,0.001598,-0.073965,0.015015,-0.001981,...,-0.105128,-0.014950,-0.039802,0.003270,-0.006049,-0.038369,15.000000,25.000000,1.000000,6.000000
50%,-0.017313,0.073738,0.049874,0.044959,-0.068874,-0.089224,0.039623,-0.034788,0.050111,0.036198,...,-0.069593,0.021222,-0.003279,0.057083,0.036988,0.004224,30.000000,27.000000,1.000000,6.000000
75%,0.015003,0.101526,0.087481,0.082385,-0.026185,-0.052010,0.076055,0.002081,0.085094,0.074090,...,-0.028467,0.053432,0.033408,0.109684,0.078949,0.044731,46.000000,30.000000,1.000000,6.000000
max,0.316810,0.519338,0.344814,0.443703,0.169458,0.298595,0.309478,0.246881,0.260366,0.270373,...,0.388045,0.215233,0.205440,0.332756,0.269439,0.385349,60.000000,61.000000,1.000000,13.000000


In [9]:
from sklearn.preprocessing import RobustScaler # eliminate outliers

x_train = {}
x_valid = {}
x_test = {}

y_train = {}
y_valid = {}
y_test = {}

#create dictionaries for each label
for target_label in ['label_1','label_2','label_3','label_4']:

  if target_label == "label_2":
    train = train_df[train_df['label_2'].notna()]
    valid = valid_df[valid_df['label_2'].notna()]
  else:
    train = train_df
    valid = valid_df

  test = test_df

  scaler = RobustScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(train.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_train[target_label] = train[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(valid.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_valid  [target_label] = valid[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(test.drop(["ID"],axis=1)), columns=[f'feature_{i}' for i in range(1,769)])

In [10]:
x_train_df = x_train['label_2'].copy()
y_train_df = y_train['label_2'].copy()

x_valid_df = x_valid['label_2'].copy()
y_valid_df = y_valid['label_2'].copy()

x_test_df = x_test['label_2'].copy()

# Cross Validation

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold

# Perform cross-validation
scores = cross_val_score(SVC(), x_train_df, y_train_df, cv=5, scoring='accuracy')

mean_accuracy = scores.mean()
std_accuracy = scores.std()
# Print the cross-validation scores
print('Support Vector Machines')
print('\n')
print("Cross-validation scores:", scores)
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")

Support Vector Machines


Cross-validation scores: [0.41155492 0.62767475 0.67475036 0.64087019 0.4775321 ]
Mean Accuracy: 0.57
Standard Deviation: 0.10


# Feature Engineering

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.975, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df))
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df))
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df))
print('Shape after PCA: ',x_train_df_pca.shape)

Shape after PCA:  (28040, 430)


In [13]:
from sklearn import metrics

# SVM

In [14]:
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

classifier = svm.SVC(kernel='linear', C=1)

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

acc_score:  0.7921195652173914


In [15]:
# from sklearn.svm import SVC
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform, randint
# import numpy as np

# param_dist = {
#     'C': [100,10,1,0,0.1,0.01],
#     'kernel': ['rbf','linear','poly','sigmoid'],
#     'gamma': ['scale','auto'],
#     'degree': [1,2,3,4],
#     'class_weight' : ['none','balanced']
# }

# svm = SVC()

# random_search = RandomizedSearchCV(
#     svm, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42, scoring='accuracy'
# )

# random_search.fit(x_train_df_pca, y_train_df)

# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

# print("best parameters:", best_params)

Label 2 
best parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'class_weight': 'balanced', 'C': 100}



In [16]:
from sklearn import svm

classifier = svm.SVC(kernel='rbf', C=100, gamma='scale', degree=4, class_weight='balanced')

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_predict_after_pca = classifier.predict(x_test_df_pca)



acc_score:  0.9483695652173914


# RandomForest

In [17]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42)

classifier.fit(x_train_df, y_train_df)

y_valid_pred = classifier.predict(x_valid_df)

print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_pred = classifier.predict(x_test_df)

accuracy_score:  0.7432065217391305


# CSV Creation

In [18]:
output_df=pd.DataFrame(columns=["ID","label_1","label_2","label_3","label_4"])

In [19]:
IDs = list(i for i in range(1, len(test_df)+1))
output_df["ID"] = IDs

In [20]:
output_df["label_2"] = y_test_predict_after_pca

In [21]:
output_df

,ID,label_1,label_2,label_3,label_4
0,1,NaN,22.0,NaN,NaN
1,2,NaN,25.0,NaN,NaN
2,3,NaN,30.0,NaN,NaN
3,4,NaN,27.0,NaN,NaN
4,5,NaN,29.0,NaN,NaN
...,...,...,...,...,...
739,740,NaN,24.0,NaN,NaN
740,741,NaN,24.0,NaN,NaN
741,742,NaN,27.0,NaN,NaN
742,743,NaN,32.0,NaN,NaN


In [22]:
output_df.to_csv('/kaggle/working/output9_2.csv',index=False)